#### Creating dataFrames for each of the raw files

In [10]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions.{col, expr}
import org.apache.spark.sql.DataFrame
import scala.util.matching 

val spark = SparkSession.builder().appName("ID2221 Project").master("local[*]").getOrCreate();
val root = "/Users/dani/Desktop/DataIntensive/";
val fileNames = Array("bbc-news-data.csv","goodreads_data.csv","job_postings.csv","mtsamples.csv");

spark = org.apache.spark.sql.SparkSession@4ea6e698
root = /Users/dani/Desktop/DataIntensive/
fileNames = Array(bbc-news-data.csv, goodreads_data.csv, job_postings.csv, mtsamples.csv)


Array(bbc-news-data.csv, goodreads_data.csv, job_postings.csv, mtsamples.csv)

In [11]:
//Schemas (not necessary but just to show what we are dealing with)
val newsSchema = StructType(Array(
  StructField("category", StringType, nullable = false),
  StructField("filename", StringType, nullable = false),
  StructField("title", StringType, nullable = false),
  StructField("content", StringType, nullable = false)
));
val bookSchema = StructType(Array(
  StructField("null", StringType, nullable = true),
  StructField("Book", StringType, nullable = true),
  StructField("Author", StringType, nullable = true),
  StructField("Description", StringType, nullable = true),
  StructField("Genres", StringType, nullable = true),
  StructField("Avg_Rating", DoubleType, nullable = true),
  StructField("Num_Ratings", IntegerType, nullable = true),
  StructField("URL", StringType, nullable = true),
));
val jobSchema = StructType(Array(
  StructField("job_id", IntegerType, nullable = true),
  StructField("company_id", IntegerType, nullable = true),
  StructField("title", StringType, nullable = true),
  StructField("description", StringType, nullable = true),
  StructField("max_salary", IntegerType, nullable = true),
  StructField("med_salary", IntegerType, nullable = true),
  StructField("min_salary", IntegerType, nullable = true),
  StructField("pay_period", StringType, nullable = true),
  StructField("formatted_work_type", StringType, nullable = true),
  StructField("location", StringType, nullable = true),
  StructField("applies", IntegerType, nullable = true),
  StructField("original_listed_time", DoubleType, nullable = true),
  StructField("remote_allowed", IntegerType, nullable = true),
  StructField("views", IntegerType, nullable = true),
  StructField("job_posting_url", StringType, nullable = true),
  StructField("application_url", StringType, nullable = true),
  StructField("application_type", StringType, nullable = true),
  StructField("expiry", DoubleType, nullable = true),
  StructField("closed_time", DoubleType, nullable = true),
  StructField("formatted_experience_level", StringType, nullable = true),
  StructField("skills_desc", StringType, nullable = true),
  StructField("listed_time", DoubleType, nullable = true),
  StructField("posting_domain", StringType, nullable = true),
  StructField("sponsored", IntegerType, nullable = true),
  StructField("work_type", StringType, nullable = true),
  StructField("currency", StringType, nullable = true),
  StructField("compensation_type", StringType, nullable = true)
));


newsSchema = StructType(StructField(category,StringType,false),StructField(filename,StringType,false),StructField(title,StringType,false),StructField(content,StringType,false))
bookSchema = StructType(StructField(null,StringType,true),StructField(Book,StringType,true),StructField(Author,StringType,true),StructField(Description,StringType,true),StructField(Genres,StringType,true),StructField(Avg_Rating,DoubleType,true),StructField(Num_Ratings,IntegerType,true),StructField(URL,StringType,true))
jobSchema = StructType(StructField(job_id,IntegerType,true),StructField(company_id,IntegerType,true),StructField(title,StringType,true),StructField(description,StringType,true),Struc...


StructType(StructField(job_id,IntegerType,true),StructField(company_id,IntegerType,true),StructField(title,StringType,true),StructField(description,StringType,true),Struc...

In [12]:
val newsData = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter","\t").option("multiLine", "true")
    .option("ignoreLeadingWhiteSpace", "true").option("ignoreTrailingWhiteSpace", "true").csv(root + "datasets/" + fileNames(0));
newsData.show(5);
//newsData.printSchema();

//Reads the second file (book data)
val bookData = spark.read.option("header", "true").option("inferSchema", "true").option("ignoreLeadingWhiteSpace", "true")
    .option("ignoreTrailingWhiteSpace", "true").option("multiLine", "true").option("escape", "\"").csv(root + "datasets/" + fileNames(1));
bookData.show(5);
//bookData.printSchema();

//Reads the third file (job postings)  (It does not seem to parse correctly but it is sufficient to show and test my function)
val jobData = spark.read.option("header", "true").option("inferSchema", "true").option("ignoreLeadingWhiteSpace", "true")
    .option("ignoreTrailingWhiteSpace", "true").option("multiLine", "true").csv(root + "datasets/job_postings/" + fileNames(2));
jobData.show(5);
//jobData.printSchema();

//Reads the fourth file (medical transcripts)
val medicalData = spark.read.option("header", "true").option("inferSchema", "true").option("ignoreLeadingWhiteSpace", "true")
                 .option("ignoreTrailingWhiteSpace", "true").option("multiLine", "true").csv(root + "datasets/" + fileNames(3));
medicalData.show(5);
//medicalData.printSchema();

+--------+--------+--------------------+--------------------+
|category|filename|               title|             content|
+--------+--------+--------------------+--------------------+
|business| 001.txt|Ad sales boost Ti...|Quarterly profits...|
|business| 002.txt|Dollar gains on G...|The dollar has hi...|
|business| 003.txt|Yukos unit buyer ...|The owners of emb...|
|business| 004.txt|High fuel prices ...|British Airways h...|
|business| 005.txt|Pernod takeover t...|Shares in UK drin...|
+--------+--------+--------------------+--------------------+
only showing top 5 rows

+---+--------------------+-------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|_c0|                Book|       Author|         Description|              Genres|          Avg_Rating|      Num_Ratings|                 URL|
+---+--------------------+-------------+--------------------+--------------------+--------------------+-----------------+----------

newsData = [category: string, filename: string ... 2 more fields]
bookData = [_c0: string, Book: string ... 6 more fields]
jobData = [job_id: string, company_id: string ... 25 more fields]
medicalData = [_c0: int, description: string ... 4 more fields]


[_c0: int, description: string ... 4 more fields]

#### Extracting and pre-processing each of the dataFrames

In [13]:
//Creates dataframe with natural language under column "news article"
val newsNL = newsData.select(newsData("content").alias("news article"));

//Creates dataframe with natural language under column "book description"
val bookNL = bookData.select(bookData("Description").alias("book description"));

//Creates dataframe with natural language under column "job posting"
val jobNL = jobData.select(jobData("description").alias("job posting"));

//Creates dataframe with natural language under column "medical transcript"
val medicalNL = medicalData.select(medicalData("transcription").alias("medical transcript"));

val dfNLArray = Array(newsNL, bookNL, jobNL, medicalNL);

newsNL = [news article: string]
bookNL = [book description: string]
jobNL = [job posting: string]
medicalNL = [medical transcript: string]
dfNLArray = Array([news article: string], [book description: string], [job posting: string], [medical transcript: string])


Array([news article: string], [book description: string], [job posting: string], [medical transcript: string])

In [15]:
import org.apache.spark.sql.functions._
val sampleSize = 2225  // Number of rows you want in the sample
val seed = 12345  // Seed for reproducibility

var index = 0;
val roughProcessedDF : Array[DataFrame] = new Array[DataFrame](4);
val regexPattern = """\-|_|\s+|\p{Punct}|^\s+|\s+$""";
val headers = Array("news article", "book description", "job posting", "medical transcript");

index=0;
//Removes punctuations and whitespace
for (df <- dfNLArray) {
    val modifiedDF = df.withColumn(headers(index), lower(regexp_replace(col(headers(index)), regexPattern, " ")));
    val cutDF = modifiedDF.sample(false, sampleSize.toDouble / modifiedDF.count, seed);
    cutDF.show(10);
    println("Number of rows:" + cutDF.count());
    roughProcessedDF(index) = cutDF;
    index+=1;
}

+--------------------+
|        news article|
+--------------------+
|quarterly profits...|
|the dollar has hi...|
|the owners of emb...|
|british airways h...|
|shares in uk drin...|
|japan s economy t...|
|the us created fe...|
|india  which atte...|
|ethiopia produced...|
|a us government c...|
+--------------------+
only showing top 10 rows

Number of rows:2225
+--------------------+
|    book description|
+--------------------+
|harry potter thin...|
|it s christmas ti...|
|librarian s note ...|
|orphaned as a chi...|
|in romeo and juli...|
| this beloved boo...|
|you can find the ...|
|   i can t explai...|
|scarlett o hara  ...|
|an award or prese...|
+--------------------+
only showing top 10 rows

Number of rows:2238
+--------------------+
|         job posting|
+--------------------+
|are you a dynamic...|
|join our growing ...|
|position  lead ed...|
| service   ancill...|
| you belong here ...|
| you belong here ...|
|who we are multi ...|
|are you a fashion...|
|meltzer li

sampleSize = 2225
seed = 12345
index = 4
roughProcessedDF = Array([news article: string], [book description: string], [job posting: string], [medical transcript: string])
regexPattern = \-|_|\s+|\p{Punct}|^\s+|\s+$
headers = Array(news article, book description, job posting, medical transcript)
index = 4


4

In [17]:
//Calculates the average length of the pre-processed natural language
var totalLength : Long = 0;
var totalCount : Long = 0;
for (df <- roughProcessedDF) {
    totalCount = totalCount + df.count();
    var length : Long = 0;
    for (row <- df.collect())
        length = length + row.mkString.length;
    totalLength = totalLength + length;
}
val averageLength = totalLength / totalCount;
println(s"The average length of the documents is $averageLength characters");

The average length of the documents is 2269 characters


totalLength = 20156063
totalCount = 8883
averageLength = 2269


2269

In [18]:
val processedDF : Array[DataFrame] = new Array[DataFrame](4);
index=0;
val maxLength = averageLength;
//Cuts the natural language which is longer than the average length
for (df <- roughProcessedDF) {
    val truncatedDF = df.withColumn(headers(index), expr(s"substring(`${headers(index)}`, 1, $maxLength)"));
    truncatedDF.show(10);
    processedDF(index) = truncatedDF;
    index+=1;
}

+--------------------+
|        news article|
+--------------------+
|quarterly profits...|
|the dollar has hi...|
|the owners of emb...|
|british airways h...|
|shares in uk drin...|
|japan s economy t...|
|the us created fe...|
|india  which atte...|
|ethiopia produced...|
|a us government c...|
+--------------------+
only showing top 10 rows

+--------------------+
|    book description|
+--------------------+
|harry potter thin...|
|it s christmas ti...|
|librarian s note ...|
|orphaned as a chi...|
|in romeo and juli...|
| this beloved boo...|
|you can find the ...|
|   i can t explai...|
|scarlett o hara  ...|
|an award or prese...|
+--------------------+
only showing top 10 rows

+--------------------+
|         job posting|
+--------------------+
|are you a dynamic...|
|join our growing ...|
|position  lead ed...|
| service   ancill...|
| you belong here ...|
| you belong here ...|
|who we are multi ...|
|are you a fashion...|
|meltzer lippe is ...|
|welcome to sensib...|
+----

processedDF = Array([news article: string], [book description: string], [job posting: string], [medical transcript: string])
index = 4
maxLength = 2269


2269

In [20]:
import org.apache.spark.sql.SaveMode.Overwrite
//Saves the dataframes as processed csv class files
processedDF(0).write.mode("overwrite").option("header","true").csv(root + "processedDatasets/" + fileNames(0));
processedDF(1).write.mode("overwrite").option("header","true").csv(root + "processedDatasets/" + fileNames(1));
processedDF(2).write.mode("overwrite").option("header","true").csv(root + "processedDatasets/" + fileNames(2));
processedDF(3).write.mode("overwrite").option("header","true").csv(root + "processedDatasets/" + fileNames(3));

//Checks that data saved correctly
for (file <- fileNames) {
    val temp = spark.read.option("header", "true").option("inferSchema", "true").csv(root + "processedDatasets/" + file);
    temp.show(5);
    println(temp.count());
}

+--------------------+
|        news article|
+--------------------+
|quarterly profits...|
|the dollar has hi...|
|the owners of emb...|
|british airways h...|
|shares in uk drin...|
+--------------------+
only showing top 5 rows

2225
+--------------------+
|    book description|
+--------------------+
|harry potter thin...|
|it s christmas ti...|
|librarian s note ...|
|orphaned as a chi...|
|in romeo and juli...|
+--------------------+
only showing top 5 rows

2200
+--------------------+
|         job posting|
+--------------------+
|are you a dynamic...|
|join our growing ...|
|position  lead ed...|
|service   ancilla...|
|you belong here  ...|
+--------------------+
only showing top 5 rows

2209
+--------------------+
|  medical transcript|
+--------------------+
|subjective   this...|
|past medical hist...|
|history of presen...|
|2 d m mode     1 ...|
|2 d echocardiogra...|
+--------------------+
only showing top 5 rows

2184


lastException = null


null

## 2. Main Function

In [ ]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame
import scala.util.Random
import org.apache.spark.sql.functions._

val spark = SparkSession.builder().appName("ID2221 Project").master("local[*]").getOrCreate();

import spark.implicits._

/* 
The function takes a sequence of pre-processed class files and combines them into an evenly balanced data set which it then 
splits into smaller sets
:param processedFiles: a string array which contains the paths to all the class files which should be considered by the function
:param relativeDist: an array of doubles which determines how the created training sets should be distributed (and how many should be created)
:param outputDirectory: a string which is the path to the output directory were the created sets will be stored
:param outputNames: an array containing the names of the output files corresponding to the distribution given by relativeDist
:param classSize: a integer specifying how many entries of each class the combined output sets should contain, by default as large as possible
*/
def createDataSets(processedFiles : Array[String], relativeDist : Array[Double], outputDirectory : String, 
                   outputNames : Array[String], classSize : Int = 0 ) {
    //Creates an array holding dataframes of each input file
    val fileDF: Array[DataFrame] = new Array[DataFrame](processedFiles.length);
    for (i <- 0 to fileDF.length-1)
        fileDF(i) = spark.read.option("header", "true").option("inferSchema", "true").csv(processedFiles(i));
    
    //Calculates desired size
    var size : Int = classSize;
    if (size <= 0) { //If user has not specified a desired size, the fuction will create a data set which is as large as possible
        var minSize = fileDF(0).count();
        for (i <- 1 to fileDF.length-1) {
            val tempSize = fileDF(i).count();
            if (tempSize < minSize) 
                minSize=tempSize;
        }
        size=minSize.asInstanceOf[Int];
    }
    
    //Take "size" random rows from each dataframe
    val sampledDF: Array[DataFrame] = new Array[DataFrame](fileDF.length);
    for (i <- 0 to sampledDF.length-1) //Takes "size" random samples from each file
        sampledDF(i) = fileDF(i).orderBy(rand()).limit(size);
    
    //Create dataframes with columns "class" and "value"
    val pairDF: Array[DataFrame] = new Array[DataFrame](sampledDF.length);
    for (i <- 0 to pairDF.length-1) {
        pairDF(i)=sampledDF(i).withColumn("class", lit(sampledDF(i).columns(0))); //Adds "class" column with value of original header
        pairDF(i)=pairDF(i).withColumnRenamed(sampledDF(i).columns(0), "value"); //Renames original header to "value"
    }
    
    //Splits the created pairs into parts according to "relativeDist"
    val splitDF: Array[Array[DataFrame]] = new Array[Array[DataFrame]](sampledDF.length);
    for (i <- 0 to splitDF.length-1) {
        //The function sorts the results based on first column after picking random elements (we reshuffle when combining)
        splitDF(i) = pairDF(i).randomSplit(relativeDist, 0); 
    }
    
    //Combines the split pairs into the desired sets (one set for each entry in "relativeDist")
    val setDF: Array[DataFrame] = new Array[DataFrame](relativeDist.length);
    for (i <- 0 to relativeDist.length-1) {
        //Combines all the class dataframes
        setDF(i) = splitDF(0)(i);
        for (ii <- 1 to splitDF.length-1)
            setDF(i) = setDF(i).union(splitDF(ii)(i));
        
        //Shuffles the result
        setDF(i) = setDF(i).orderBy(rand());
    }
    
    //Shows some samples from each created set
    for (v <- setDF) {
        v.show(10);
        println(v.count());
    }
    
    //Saves the created sets as csv files
    for (i <- 0 to setDF.length-1)
        setDF(i).write.mode("overwrite").option("header","true").csv(outputDirectory + outputNames(i));
}

In [ ]:
val rootProcessed : String = "hdfs://localhost:9000/processedDatasets/";
val processedFiles : Array[String] = Array(rootProcessed + "bbc-news-data.csv", rootProcessed + "goodreads_data.csv", 
                                           rootProcessed + "job_postings.csv", rootProcessed + "mtsamples.csv");
// In our project we used three set, 80% train, 15% validation and 5% test
val dist : Array[Double] = Array(160, 30, 10); //Any values work, function only look at comparative sizes

val outputNames : Array[String] = Array("train", "val", "test");

createDataSets(processedFiles, dist, "hdfs://localhost:9000/project/dataSets/", outputNames);

## 3. Machine Learning on created data sets

In [ ]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.{IDF,HashingTF, Tokenizer}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}

//Loads the created data sets
val train = spark.read.option("header", "true").option("inferSchema", "true").csv(root + "dataSets/train");
val test = spark.read.option("header", "true").option("inferSchema", "true").csv(root + "dataSets/test");

//Adding column which maps classes to integers to allow for spark ml library processing
val mapper = spark.createDataFrame(Seq(("book description", 1.0),("news article", 2.0),("medical transcript", 3.0)
                                       ,("job posting", 4.0))).toDF("class", "label");
val trainSet = train.join(mapper, "class");
val testSet = test.join(mapper, "class");

//Number of words
val totalSet = train.union(test);
val nWords = totalSet.select("value").flatMap(_.toString.split(" ")).distinct().count();

//Creating the pipeline for a simple classifier
val tokenizer = new Tokenizer().setInputCol("value").setOutputCol("words");
val hashTF = new HashingTF().setInputCol(tokenizer.getOutputCol).setOutputCol("hashTF").setNumFeatures(nWords.asInstanceOf[Int]);
val tf_idf = new IDF().setInputCol(hashTF.getOutputCol).setOutputCol("tf_idf").setMinDocFreq(5);
val model = new LogisticRegression().setMaxIter(20).setRegParam(0.01).setLabelCol("label").setFeaturesCol(tf_idf.getOutputCol)
    .setFamily("multinomial");
val pipeline = new Pipeline().setStages(Array(tokenizer, hashTF, tf_idf, model));

//Sets upp hyper parameter tuning using cross-validation
val evaluator = new MulticlassClassificationEvaluator().setPredictionCol("prediction")
val paramGrid = new ParamGridBuilder().addGrid(model.regParam, Array(0.03, 0.01, 0.1)).addGrid(model.maxIter, Array(3,10,30)).build();
val crossValidator = new CrossValidator().setEstimator(pipeline).setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid).setNumFolds(4).setParallelism(2);
    
//Train the model
val trainedModel = pipeline.fit(trainSet);         //Set parameters (Fast with good performance)
//val trainedModel = crossValidator.fit(trainSet); //Simple hyper parameter tuning using cross-validation (Slow with slightly better performance)


In [ ]:
//Evaluate performance of the model
val accuracy = evaluator.evaluate(trainedModel.transform(testSet));
println(s"The trained model had an accuracy of $accuracy");

In [ ]:
//Save the trained model on disk
trainedModel.write.overwrite().save(root + "model/trainedLogisticRegressionModel");

## JUST TRYING TO CONNECT HDFS

In [ ]:

index=0
for (df <- processedDF) {
    val hdfsOutputPath = "hdfs://54.211.123.215:9000/processedFiles/"+fileNames(index)
// Assuming 'yourDataFrame' is your DataFrame with a column containing strings
    df.write
      .format("csv")
      .option("header", "true") // If your CSV has a header
      .save(hdfsOutputPath)
    index+=1
}

In [ ]:
// Import necessary libraries
import org.apache.spark.sql._

// Define the JDBC connection properties
val jdbcUrl = "jdbc:mysql://your_mysql_host:your_mysql_port/your_database"
val connectionProperties = new java.util.Properties()
connectionProperties.setProperty("user", "your_username")
connectionProperties.setProperty("password", "your_password")

// Define your DataFrame
for (df <- processedDF) {
    df.write.mode(SaveMode.Overwrite) // Choose the save mode you prefer
      .jdbc(jdbcUrl, "your_table_name", connectionProperties)
    
}